Sudoku solver using simulated annealing

In [139]:
import random
import numpy as np
from collections import Counter
from copy import deepcopy
import math
import matplotlib.pyplot as plt

size = 9

def print_sudoku(sudoku):
    for i in sudoku:
        print(i)


def generate_random_point(sudoku,row,col):
        while True:
            x, y = random.randint(row,row+2), random.randint(col, col+2)
            if sudoku[x][y]==0:
                return x, y

def read_file(path):
    lines = []
    with open(str(path), "r") as f:
        lines = f.readlines()
    sudoku = [[0]*size for _ in range(size)]



    for row, line in enumerate(lines):
        for col, char in enumerate(line):

            if char != 'x' and char != '\n':
                sudoku[row][col] = int(char)

    original = deepcopy(sudoku)
    for i in range(0,size, +3):
        for j in range(0,size, +3):
            generate_small_square(sudoku, i, j)
    return sudoku, original


def calc_cost(sudoku):

    cost = 0
    for i in range(size):
        cntrows = [0]*size
        cntcols = [0]*size
        for j in range(size):
            cntrows[sudoku[i][j]-1] += 1
            cntcols[sudoku[j][i]-1] += 1
        cost += sum(x>1 for x in cntrows) + sum(y>1 for y in cntcols)
    return cost

def swap(sudoku, p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]

    sudoku[x1][y1], sudoku[x2][y2] = sudoku[x2][y2], sudoku[x1][y1]


def generate_next_state(original, sudoku,squareNums):

    square_to_change = random.randint(0,8)
    x, y = squareNums.get(square_to_change)

    p1x, p1y = generate_random_point(original, x, y)
    p2x, p2y = generate_random_point(original, x, y)
    while (p2x, p2y) == (p1x, p1y):
        p2x, p2y = generate_random_point(original, x, y)
    
    swap(sudoku, (p1x,p1y), (p2x,p2y))

    return (p1x,p1y), (p2x,p2y)

    


def generate_small_square(sudoku, row, col):

    existing_elements = []

    

    for i in range(row,row+3,1):
        for j in range(col, col+3, 1):
            if sudoku[i][j] != 0:
                existing_elements.append(sudoku[i][j])
    
    while len(existing_elements)<9:
        randNum = random.randint(1,9)
        if not randNum in existing_elements:
            x, y = generate_random_point(sudoku,row,col)

            sudoku[x][y] = randNum
            existing_elements.append(randNum)


def calcDeviation(original, sudoku, squareNums):
    
    tmp = []
    for i in range(200):
        tmp.append(calc_cost(sudoku))
        generate_next_state(original, sudoku, squareNums)

    return np.std(tmp)


def probability(new,old,T):
    if(T==0):
        return 0.0
    try:
        return math.exp(-(new-old)/T)
    except OverflowError:
        return 0.0

def simulated_annealing(path, max_iterations = 1e6,temperature=lambda i: 0.99999**i):
    sudoku, original = read_file(path)
    

    #key = number of square, value = column index, row index
    squareNumbers = {0: (0,0), 1: (0,3), 2: (0,6),
                    3: (3,0), 4: (3,3), 5: (3,6),
                    6: (6,0), 7: (6,3),8: (6,6)} 

    # T = calcDeviation(original, sudoku, squareNumbers)

    print_sudoku(sudoku)
    print(calc_cost(sudoku))

    sudoku_cost = calc_cost(sudoku)


    for i in range(int(max_iterations)):

        T = temperature(i)
        p1,p2 = generate_next_state(original,sudoku,squareNumbers)
        new_cost = calc_cost(sudoku)

        if new_cost == 0:
            print('solved')
            print_sudoku(sudoku)
            break
        elif new_cost<sudoku_cost or random.random() < probability(new_cost,sudoku_cost,T):
            sudoku_cost = new_cost
        else:
            swap(sudoku, p1, p2)


In [140]:
simulated_annealing('mid1.txt')


[6, 2, 5, 3, 7, 8, 4, 9, 5]
[3, 8, 7, 4, 2, 9, 8, 7, 1]
[9, 4, 1, 5, 6, 1, 3, 6, 2]
[8, 4, 6, 1, 8, 2, 3, 7, 5]
[2, 7, 9, 7, 9, 6, 1, 6, 9]
[1, 5, 3, 5, 3, 4, 2, 4, 8]
[8, 7, 5, 1, 3, 7, 5, 2, 4]
[9, 3, 1, 9, 6, 8, 6, 3, 9]
[2, 4, 6, 2, 5, 4, 8, 1, 7]
39
solved
[9, 6, 2, 3, 7, 8, 4, 5, 1]
[1, 8, 5, 4, 2, 9, 7, 3, 6]
[3, 7, 4, 5, 6, 1, 9, 8, 2]
[4, 9, 6, 8, 3, 2, 1, 7, 5]
[2, 1, 8, 7, 4, 5, 3, 6, 9]
[7, 5, 3, 1, 9, 6, 2, 4, 8]
[8, 2, 7, 6, 1, 3, 5, 9, 4]
[5, 3, 1, 9, 8, 4, 6, 2, 7]
[6, 4, 9, 2, 5, 7, 8, 1, 3]


In [141]:
simulated_annealing('easy1.txt')


[1, 2, 3, 4, 7, 6, 1, 2, 3]
[7, 5, 6, 2, 3, 8, 9, 7, 5]
[4, 8, 9, 5, 1, 9, 8, 4, 6]
[5, 1, 8, 6, 3, 9, 2, 3, 7]
[3, 6, 4, 7, 8, 2, 6, 4, 1]
[9, 7, 2, 1, 5, 4, 5, 9, 8]
[2, 8, 4, 5, 9, 7, 3, 1, 2]
[7, 5, 1, 1, 2, 4, 4, 8, 6]
[6, 9, 3, 8, 3, 6, 7, 9, 5]
34
solved
[8, 2, 1, 4, 7, 6, 9, 5, 3]
[9, 5, 6, 2, 3, 8, 1, 7, 4]
[4, 3, 7, 1, 9, 5, 8, 6, 2]
[5, 1, 8, 6, 4, 9, 2, 3, 7]
[3, 6, 9, 7, 8, 2, 5, 4, 1]
[7, 4, 2, 3, 5, 1, 6, 9, 8]
[2, 8, 4, 5, 6, 7, 3, 1, 9]
[1, 7, 5, 9, 2, 3, 4, 8, 6]
[6, 9, 3, 8, 1, 4, 7, 2, 5]
